# Multiclass Classification - Iris data

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.functional as F
import torch.optim as optim
import numpy as np
import random

random.seed(777)
torch.manual_seed(777)

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## Load Data & Preprocessing

In [9]:
iris = load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [4]:
train_X, test_X, train_y, test_y = train_test_split(iris.data, iris.target, test_size=0.3)

In [5]:
train_y

array([2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 1, 2, 2, 1, 0, 1, 1, 2, 2, 2, 2, 0,
       1, 0, 2, 0, 1, 2, 0, 2, 2, 0, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 0,
       1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 2, 2, 0, 0,
       1, 2, 0, 2, 1, 0, 0, 1, 1, 0, 1, 1, 2, 0, 0, 1, 0, 1, 2, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 2, 1, 0, 2, 2, 2, 2, 0, 2, 0, 1])

In [6]:
class makeData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [7]:
train_data = makeData(np.array(normalize(train_X)), np.array(train_y)) #normalize은 기본적으로 열(column)기준이다.
test_data = makeData(np.array(normalize(test_X)), np.array(test_y))

In [8]:
train_data.__getitem__(0)

(array([0.69589887, 0.34794944, 0.57629125, 0.25008866]), 2)

In [9]:
train_data.__len__()

105

## Modeling

In [19]:
class Multiclass_Classification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(Multiclass_Classification, self).__init__()
        
        self.FC1 = nn.Sequential(
                                nn.Linear(num_feature, 100),
                                nn.ReLU()
                            )
        self.FC2 = nn.Sequential(
                                nn.Linear(100, 50),
                                nn.ReLU()
                            )
        
        self.FC3 = nn.Sequential(
                                nn.Linear(50, 30),
                                nn.ReLU()
                            )
        self.dropout = nn.Dropout(0.5)
        
        self.out = nn.Linear(30, num_class)
        
    def forward(self, inputs):
        x = self.FC1(inputs)
        x = self.FC2(x)
        x = self.FC3(x)
        x = self.dropout(x)
        x = self.out(x)
        return x
    
    def predict(self, test_inputs):
        x = self.FC1(inputs)
        x = self.FC2(x)
        x = self.FC3(x)
        x = self.out(x)
        return torch.max(x, 1)[1]

## Training

In [25]:
EPOCHS = 2000
BATCH_SIZE = 20
NUM_FEATURE = 4
NUM_CLASS = 3

model = Multiclass_Classification(NUM_FEATURE, NUM_CLASS)
criterion = nn.CrossEntropyLoss() #pytorch에서는 NLL loss + log softmax가 이미 적용되어 있다
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=0.0007)

for epoch in range(EPOCHS):
    for X_batch, y_batch in train_loader:
        inputs = Variable(torch.Tensor(X_batch.float()))
        targets = Variable(torch.LongTensor(y_batch.long())) #torch.tensor는 float로 들어간다.
        model.zero_grad()
        y_pred = model(inputs)
        loss = criterion(y_pred, targets)
        loss.backward()
        optimizer.step()
    
    if epoch % 200 == 0:
        print(loss)

tensor(1.1074, grad_fn=<NllLossBackward>)
tensor(0.0223, grad_fn=<NllLossBackward>)
tensor(0.0045, grad_fn=<NllLossBackward>)
tensor(0.1829, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.0016, grad_fn=<NllLossBackward>)
tensor(0.0001, grad_fn=<NllLossBackward>)
tensor(0.1670, grad_fn=<NllLossBackward>)
tensor(0.0596, grad_fn=<NllLossBackward>)
tensor(0.0935, grad_fn=<NllLossBackward>)


## Test Accuracy

In [26]:
test_loader = DataLoader(dataset=test_data, batch_size=1)
test_y_true = []
test_y_pred = []

for X_batch, y_batch in test_loader:
    inputs = Variable(torch.Tensor(X_batch.float()))
    test_y_true.append(y_batch.item())
    y_pred = model.predict(inputs)
    test_y_pred.append(int(y_pred.item()))

In [27]:
print(test_y_pred)

[1, 1, 2, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 2, 1, 1, 2, 1, 2, 2, 0, 2, 0, 1, 0, 0, 1, 2, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 2, 2, 0]


In [28]:
print(test_y_true)

[1, 1, 2, 0, 1, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 2, 1, 1, 2, 1, 2, 2, 0, 2, 0, 1, 0, 0, 2, 2, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 2, 2, 0]


In [29]:
accuracy_score(test_y_pred, test_y_true)

0.9555555555555556

# Multi-Classification_wine_quality

In [1]:
import pandas as pd

In [4]:
wine = pd.read_csv('./5_winequality-red.csv', sep=';')

In [12]:
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [32]:
train_X, test_X, train_y, test_y = train_test_split(wine.iloc[:,range(11)], wine.iloc[:,-1], test_size=0.3)

In [33]:
class makeData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [34]:
train_data = makeData(np.array(normalize(train_X)), np.array(train_y)) #normalize은 기본적으로 열(column)기준이다.
test_data = makeData(np.array(normalize(test_X)), np.array(test_y))

In [35]:
train_data.__getitem__(0)

(array([0.4084516 , 0.03664051, 0.00240266, 0.09009962, 0.00342379,
        0.30033206, 0.60066412, 0.0597811 , 0.20542713, 0.03603985,
        0.57063091]), 5)

In [24]:
train_data.__len__()

1119

In [40]:
class Multiclass_Classification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(Multiclass_Classification, self).__init__()
        
        self.FC1 = nn.Sequential(
                                nn.Linear(num_feature, 10),
                                nn.ReLU()
                            )
        self.FC2 = nn.Sequential(
                                nn.Linear(10, 5),
                                nn.ReLU()
                            )
        
        self.FC3 = nn.Sequential(
                                nn.Linear(5, 3),
                                nn.ReLU()
                            )
        self.dropout = nn.Dropout(0.9)
        
        self.out = nn.Linear(3, num_class)
        
    def forward(self, inputs):
        x = self.FC1(inputs)
        x = self.FC2(x)
        x = self.FC3(x)
        x = self.dropout(x)
        x = self.out(x)
        return x
    
    def predict(self, test_inputs):
        x = self.FC1(inputs)
        x = self.FC2(x)
        x = self.FC3(x)
        x = self.out(x)
        return torch.max(x, 1)[1]

In [41]:
EPOCHS = 2000
BATCH_SIZE = 20
NUM_FEATURE = 11
NUM_CLASS = 11

model = Multiclass_Classification(NUM_FEATURE, NUM_CLASS)
criterion = nn.CrossEntropyLoss() #pytorch에서는 NLL loss + log softmax가 이미 적용되어 있다
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

optimizer = optim.Adam(model.parameters(), lr=0.0007)

for epoch in range(EPOCHS):
    for X_batch, y_batch in train_loader:
        inputs = Variable(torch.Tensor(X_batch.float()))
        targets = Variable(torch.LongTensor(y_batch.long())) #torch.tensor는 float로 들어간다.
        model.zero_grad()
        y_pred = model(inputs)
        loss = criterion(y_pred, targets)
        loss.backward()
        optimizer.step()
    
    if epoch % 200 == 0:
        print(loss)

tensor(2.2388, grad_fn=<NllLossBackward>)
tensor(0.8726, grad_fn=<NllLossBackward>)
tensor(1.4144, grad_fn=<NllLossBackward>)
tensor(1.0713, grad_fn=<NllLossBackward>)
tensor(1.3406, grad_fn=<NllLossBackward>)
tensor(1.0839, grad_fn=<NllLossBackward>)
tensor(1.0273, grad_fn=<NllLossBackward>)
tensor(1.0151, grad_fn=<NllLossBackward>)
tensor(1.0841, grad_fn=<NllLossBackward>)
tensor(1.1552, grad_fn=<NllLossBackward>)


In [42]:
test_loader = DataLoader(dataset=test_data, batch_size=1)
test_y_true = []
test_y_pred = []

for X_batch, y_batch in test_loader:
    inputs = Variable(torch.Tensor(X_batch.float()))
    test_y_true.append(y_batch.item())
    y_pred = model.predict(inputs)
    test_y_pred.append(int(y_pred.item()))

In [43]:
accuracy_score(test_y_pred, test_y_true)

0.37916666666666665